In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.conv_layer(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layer(x)
        return x

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)

model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(10):
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/10], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')


100%|██████████| 9912422/9912422 [00:00<00:00, 215270961.02it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 85608264.19it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 116258869.95it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 4160412.48it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch [1/10], Step [100/938], Loss: 0.2011
Epoch [1/10], Step [200/938], Loss: 0.2580
Epoch [1/10], Step [300/938], Loss: 0.0832
Epoch [1/10], Step [400/938], Loss: 0.1164
Epoch [1/10], Step [500/938], Loss: 0.0263
Epoch [1/10], Step [600/938], Loss: 0.0748
Epoch [1/10], Step [700/938], Loss: 0.0068
Epoch [1/10], Step [800/938], Loss: 0.0596
Epoch [1/10], Step [900/938], Loss: 0.0570
Epoch [2/10], Step [100/938], Loss: 0.0103
Epoch [2/10], Step [200/938], Loss: 0.0714
Epoch [2/10], Step [300/938], Loss: 0.0036
Epoch [2/10], Step [400/938], Loss: 0.0722
Epoch [2/10], Step [500/938], Loss: 0.0697
Epoch [2/10], Step [600/938], Loss: 0.0606
Epoch [2/10], Step [700/938], Loss: 0.0730
Epoch [2/10], Step [800/938], Loss: 0.0163
Epoch [2/10], Step [900/938], Loss: 0.0025
Epoch [3/10], Step [100/938], Loss: 0.0038
Epoch [3/10], Step [200/938], Loss: 0.0008
Epoch [3/10], Step [300/938], Loss: 0.0067
Epoch [3/10], Step [40